In [1]:
import math 
import string 
import re

import json 

import pandas as pd 
import sqlite3



create_table_generated_summaries_str =  '''CREATE TABLE generated_summaries (
    uuid INTEGER PRIMARY KEY AUTOINCREMENT, 
    docid TEXT NOT NULL ,
    model TEXT NOT NULL, 
    summary TEXT NOT NULL, 
    dialogue TEXT
);'''

create_table_label_str = '''CREATE TABLE label (
    uuid INTEGER PRIMARY KEY AUTOINCREMENT, 
    user_id TEXT NOT NULL,
    docid TEXT NOT NULL,
    model TEXT NOT NULL,
    nonfactual_spans ENUM ,
    evidence TEXT ,
    error_type TEXT,
    summary TEXT NOT NULL,
    dialogue TEXT
);'''




 
# filter_ids = ["'" + each + "'" for each in filter_ids]
# str_filter_ids = '(' + ', '.join(filter_ids) + ')'

def connect_to_db(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    return conn, c 


def create_table(create_str, db_path):
    conn, c = connect_to_db(db_path)
    c.execute('''%s'''%(create_str))
    conn.commit()
    conn.close()

def add_data(filename, db_path):
    df = pd.read_csv(filename)
    print(len(df))
    conn, c = connect_to_db(db_path)
#     print('Size before adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
    for idx, row in df.iterrows():
        # print(row)
        docid = row['DocID']
        dialogue = row['Dialogue']
        model = row['Model']
        origin = row['origin']
        summary = row['Summary']
        
        if 'Summary' in df.keys() and row['DocID'] in filter_ids:
            generic_summary = row['Summary']
            summary_uuid_generic = f'{docid}_{model}_gen'
            c.execute("""INSERT INTO generated_summaries (docid, model, summary, dialogue) VALUES (?, ?, ?, ?)""",
                                                        (docid, 
                                                        model,
                                                        summary,
                                                        dialogue))
            
            
    
    print('Size after adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
    conn.commit()
    conn.close()
    
def create_tables(db_path, force_new):
    print(force_new)
    conn, c = connect_to_db(db_path)
    create_cmds = {
        'generated_summaries': create_table_generated_summaries_str,
        'label': create_table_label_str
    }
    
    for table_name, create_str in create_cmds.items():
        table_results = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';").fetchall()
        print(table_results)
        if len(table_results) != 0:
            print('Size before adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
            if (force_new == True):
                print('here', force_new)
                print(f'Dropping {table_name} ...')
                c.execute(f'DROP table {table_name};')
                create_table(create_str, db_path)
        else:
             create_table(create_str, db_path)

In [2]:
database_path = '/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/annotations/gpt4_summaries_short.db'
conn, c = connect_to_db(database_path)

In [6]:


results = c.execute(f"""SELECT * FROM label WHERE user_id == 'elisa'""").fetchall()



In [7]:
results[0]

(6,
 'elisa',
 'test_12',
 'GPT4',
 'Person 1 (Steven) noticed that Pe',
 'What happene',
 'Intrinsic_Error',
 "Person 1 (Steven) noticed that Person 2 (Tony) looked unhappy. Tony admitted making a big mistake, revealing he got caught cheating on a test which made him feel ashamed. He explains his fear of failing due to his father's expectations. However, Steven advises him to study hard and learn from his mistakes.",
 "#Person1#: Hi, Tony. You look unhappy. What's wrong?\r\n#Person2#: Oh, Steven, I made a big mistake.\r\n#Person1#: What happened?\r\n#Person2#: I really wish I hadn't done it.\r\n#Person1#: What on earth are you talking about?\r\n#Person2#: I got caught cheating. I feel so ashamed. The teacher saw me and told me I failed.\r\n#Person1#: What were you thinking?\r\n#Person2#: You know my father. If I fail, he'll kill me. I have to do well.\r\n#Person1#: But what you should do is study hard.\r\n#Person2#: I know. . . I know. . . it's all my fault. I feel awful that I didn't